In [26]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [27]:
# download packages

from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import tensorflow as tf
import os
import cv2

import numpy as np
from glob import glob

In [8]:
# connect with the google drive
#from google.colab import drive
# drive.mount('/content/drive')

In [9]:
# read the file from google drive
#plantdataset='/content/drive/MyDrive/Data Science/PlantDataset2'
#plantdataset

'/content/drive/MyDrive/Data Science/PlantDataset2'

Because this dataset has been divided into three sets: training, testing, and validation. So we don't need to split dataset into traning,testing and validation set.



In [10]:
#train_set='/content/drive/MyDrive/Data Science/PlantDataset2/Train'
#train_set

'/content/drive/MyDrive/Data Science/PlantDataset2/Train'

In [11]:
#test_set='/content/drive/MyDrive/Data Science/PlantDataset2/Test'
#test_set

'/content/drive/MyDrive/Data Science/PlantDataset2/Test'

In [12]:
#val_set='/content/drive/MyDrive/Data Science/PlantDataset2/Validation'
#val_set

'/content/drive/MyDrive/Data Science/PlantDataset2/Validation'

In [29]:
# read the image file

dataset_url="C:/Users/Laura/Documents/Data Science/UoM/Machine Learning COMP7745/project/MachineLearningProject/dataset"



# New Section

In [39]:
from zmq.constants import CHANNEL
# set up the size value of image and batch
IMAGE_SIZE=(4000,2672) 
BATCH_SIZE=50
CHANNEL=3
EPOCHS=20

In [40]:
# create a images datasets
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "C:/Users/Laura/Documents/Data Science/UoM/Machine Learning COMP7745/project/MachineLearningProject/dataset/train",
    shuffle=True,
    image_size=(4000,2672),
    batch_size=BATCH_SIZE
)


Found 1532 files belonging to 3 classes.


In [41]:
class_names=dataset.class_names
class_names

['Test', 'Train', 'Validation']

In [ ]:
for image_batch, label_batch in dataset.take(1): 
  print(image_batch.shape)
  print(label_batch.numpy()) 

In [ ]:
for image_batch, label_batch in dataset.take(1):
  print(image_batch[0].shape)

In [ ]:
for image_batch, label_batch in dataset.take(1):
  print(image_batch[0].numpy())

In [ ]:
# visulaize the first 16 images of training set
for image_batch, label_batch in dataset.take(1):
  for i in range(16):
    ax=plt.subplot(4,4,i+1)   #4 rows and 4 columns
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    plt.title(class_names[label_batch[i]])
    plt.axis("off")

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"), #creates a layer that randomly flips the input images horizontally and/or vertically with a probability of 0.5.
    layers.experimental.preprocessing.RandomRotation(0.2)    #creates a layer that randomly rotates the input images by a factor of up to 0.2 radians (about 11.5 degrees) in either direction.
])

In [ ]:
from keras.api._v2.keras import activations
# build a model
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNEL)
n_classes=3

model=models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=(4000, 2672,3)),   # lenth, width, color
    #data_augmentation,
    layers.Conv2D(32,(3,3),activation="relu",input_shape=input_shape),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation="relu"),
    layers.Dense(n_classes,activation="softmax"),

]
    
)

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history=model.fit(
    train_set,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_set
 
)

In [ ]:
scores=model.evaluate(test_set)
scores

In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
history.history['accuracy']

In [ ]:
accuracy=history.history['accuracy']
validation_accuracy=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
# visulaize the training and validation accuracy 
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),accuracy,label='Training Accuracy')
plt.plot(range(EPOCHS),validation_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Trianing and Validation Accuracy')
plt.show()

In [ ]:
# visulaize the loss of training and validation 
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),loss,label='Training Loss')
plt.plot(range(EPOCHS),val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Trianing and Validation Loss')
plt.show()

In [ ]:
# save model
import pickle
with open('model_plant_CNN','wb') as f:
  pickle.dump(model,f)
